# Importing Libraries

In [ ]:
import numpy as np 
import json
from matplotlib import pyplot as plt
from skimage import color
from skimage.feature import hog
from sklearn import svm
from sklearn.metrics import classification_report,accuracy_score



# Data Pre-processing

In [ ]:
f = open(r"/content/shipsnet.json")
dataset = json.load(f)
f.close()

dataset.keys()

FileNotFoundError: ignored

In [ ]:
data = np.array(dataset['data']).astype('uint8')
img_length = 80
data = data.reshape(-1,3,img_length,img_length).transpose([0,2,3,1])

plt.imshow(data[51])

NameError: ignored

In [ ]:
data_gray = [ color.rgb2gray(i) for i in data]
plt.imshow(data_gray[51])

# HOG Feature Extraction

In [ ]:
ppc = 16
hog_images = []
hog_features = []
for image in data_gray:
    fd,hog_image = hog(image, orientations=8, pixels_per_cell=(ppc,ppc),cells_per_block=(4, 4),block_norm= 'L2',visualize=True)
    hog_images.append(hog_image)
    hog_features.append(fd)

NameError: ignored

In [ ]:
plt.imshow(hog_images[51])

IndexError: ignored

In [ ]:
labels =  np.array(dataset['labels']).reshape(len(dataset['labels']),1)

# SV Classifier

In [ ]:
clf = svm.SVC()
hog_features = np.array(hog_features)
data_frame = np.hstack((hog_features,labels))
np.random.shuffle(data_frame)

In [ ]:
percentage = 80
# percentage of training data 
partition = int(len(hog_features)*percentage/100)

In [ ]:
x_train, x_test = data_frame[:partition,:-1],  data_frame[partition:,:-1]
y_train, y_test = data_frame[:partition,-1:].ravel() , data_frame[partition:,-1:].ravel()

clf.fit(x_train,y_train)

In [ ]:
y_pred = clf.predict(x_test)

In [ ]:
print("Accuracy: "+str(accuracy_score(y_test, y_pred)))
print('\n')
print(classification_report(y_test, y_pred))


# Neural Network Model


In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf

In [ ]:
df = pd.read_json("/content/shipsnet.json");

In [ ]:
# Normalize and reshape the image data
df["normalized_data"] = df["data"].apply(lambda x: (np.array(x) / 255).reshape(80, 80, 3))

In [ ]:
# Define X and Y
X = df["normalized_data"]
Y = df["labels"]
# Split the data into training and testing sets. Use a 75/25 split
from sklearn.model_selection import train_test_split
(X_train, X_test, Y_train, Y_test) = train_test_split(X, Y, test_size=0.25, random_state=42)

In [ ]:
X_train = np.array([x for x in X_train])
X_test = np.array([x for x in X_test])
Y_train = np.array([y for y in Y_train])
Y_test = np.array([y for y in Y_test])

In [ ]:
from tensorflow.keras import datasets, layers, models
from tensorflow.keras.layers import Activation
# Starts the model with a sequential ANN
model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(80, 80, 3)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(1))
model.add(Activation('sigmoid'))

In [ ]:
model.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=['accuracy'])

In [ ]:
gen_model = model.fit(X_train, Y_train, epochs=10, validation_data=(X_test, Y_test))

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix
predictions = model.predict(X_test)
print(classification_report(Y_test, predictions.round()))
print(confusion_matrix(Y_test, predictions.round()))